In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# Create engine using the `covid.sqlite` database file
engine = create_engine("sqlite:///covid.sqlite")

In [3]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

covid
-----------
index BIGINT
Serial Number BIGINT
Country TEXT
Total Cases INTEGER
Total Deaths INTEGER
Total Recovered INTEGER
Active Cases INTEGER
Total Test INTEGER
Population INTEGER
Continent TEXT
country_code TEXT
latitude FLOAT
longitude FLOAT
TTest_TCases_Ratio FLOAT
TTest_TDeath_Ratio FLOAT
TTest_Pop_Ratio FLOAT
TCases_TReco_Ratio FLOAT



In [4]:
# # user inputs
# user_region = 'All'
# user_min_attempts = 0

# # switch on user_region
# if user_region == 'All':
#     where_clause = "and 1=1"
# else:
#     where_clause = f"and region = '{user_region}'"

# build the query
query = f"""
    SELECT
        *
    FROM
        covid
        ;
"""

# execute query
df = pd.read_sql(text(query), con=engine)
df.head(10)

,index,Serial Number,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population,Continent,country_code,latitude,longitude,TTest_TCases_Ratio,TTest_TDeath_Ratio,TTest_Pop_Ratio,TCases_TReco_Ratio
0,0,1,United States,104196861,1132935,101322779,1741147,1159832679,334805269,North America,US,37.090240,-95.712891,11.13,1023.74,3.46,1.03
1,1,2,India,44682784,530740,44150289,1755,915265788,1406631776,Asia,IN,20.593684,78.962880,20.48,1724.51,0.65,1.01
2,2,3,France,39524311,164233,39264546,95532,271490188,65584518,Europe,FR,46.227638,2.213749,6.87,1653.08,4.14,1.01
3,3,4,Germany,37779833,165711,37398100,216022,122332384,83883596,Europe,DE,51.165691,10.451526,3.24,738.23,1.46,1.01
4,4,5,Brazil,36824580,697074,35919372,208134,63776166,215353593,South America,BR,-14.235004,-51.925280,1.73,91.49,0.30,1.03
5,5,6,Japan,32588442,68399,21567425,10952618,92144639,125584838,Asia,JP,36.204824,138.252924,2.83,1347.16,0.73,1.51
6,6,7,S. Korea,30197066,33486,29740877,422703,15804065,51329899,Asia,KR,35.907757,127.766922,0.52,471.96,0.31,1.02
7,7,8,Italy,25453789,186833,25014986,251970,265478247,60262770,Europe,IT,41.871940,12.567380,10.43,1420.94,4.41,1.02
8,8,9,UK,24274361,204171,24020088,50102,522526476,68497907,Europe,GB,55.378051,-3.435973,21.53,2559.26,7.63,1.01
9,9,10,Russia,21958696,395108,21356008,207580,273400000,145805947,Asia,RU,61.524010,105.318756,12.45,691.96,1.88,1.03


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               197 non-null    int64  
 1   Serial Number       197 non-null    int64  
 2   Country             197 non-null    object 
 3   Total Cases         197 non-null    int64  
 4   Total Deaths        197 non-null    int64  
 5   Total Recovered     197 non-null    int64  
 6   Active Cases        197 non-null    int64  
 7   Total Test          197 non-null    int64  
 8   Population          197 non-null    int64  
 9   Continent           197 non-null    object 
 10  country_code        185 non-null    object 
 11  latitude            195 non-null    float64
 12  longitude           196 non-null    float64
 13  TTest_TCases_Ratio  197 non-null    float64
 14  TTest_TDeath_Ratio  197 non-null    float64
 15  TTest_Pop_Ratio     197 non-null    float64
 16  TCases_T

In [6]:
df.describe()

,index,Serial Number,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population,latitude,longitude,TTest_TCases_Ratio,TTest_TDeath_Ratio,TTest_Pop_Ratio,TCases_TReco_Ratio
count,197.000000,197.000000,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,195.000000,196.000000,197.000000,197.000000,197.000000,197.000000
mean,105.619289,106.619289,3.384525e+06,3.398597e+04,3.251507e+06,9.903209e+04,3.505034e+07,3.934992e+07,19.594188,13.960951,14.160863,3307.711929,2.069289,1.246041
std,64.350778,64.350778,1.019192e+07,1.120744e+05,9.845320e+06,7.947910e+05,1.220900e+08,1.471369e+08,23.781978,67.306978,27.547566,10635.417132,3.586711,1.146081
min,0.000000,1.000000,1.403000e+03,1.000000e+00,4.380000e+02,0.000000e+00,7.850000e+03,4.965000e+03,-40.900557,-177.156097,0.520000,33.550000,0.010000,1.000000
25%,50.000000,51.000000,3.824000e+04,3.140000e+02,3.636600e+04,8.200000e+01,4.016220e+05,1.184817e+06,5.787949,-15.212844,5.670000,475.590000,0.180000,1.010000
50%,102.000000,103.000000,3.292270e+05,3.164000e+03,3.259340e+05,1.319000e+03,2.697814e+06,7.040745e+06,18.109581,18.687854,8.420000,901.080000,0.890000,1.020000
75%,160.000000,161.000000,1.746997e+06,1.692600e+04,1.731007e+06,1.174000e+04,1.580406e+07,2.917808e+07,40.106102,46.366989,14.130000,1998.340000,2.360000,1.040000
max,224.000000,225.000000,1.041969e+08,1.132935e+06,1.013228e+08,1.095262e+07,1.159833e+09,1.412000e+09,71.706936,179.414413,317.900000,109701.620000,23.300000,10.020000


In [7]:
query = """
            SELECT 
                Population,
                "Total Cases",
                "Active Cases",
                "Total Recovered",
                Continent,
                latitude,
                longitude
                        
            From 
                covid
            Group by 
                 Continent
            Order by 
                "Population" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head()
# df3.info()

,Population,Total Cases,Active Cases,Total Recovered,Continent,latitude,longitude
0,1406631776,44682784,1755,44150289,Asia,20.593684,78.962880
1,334805269,104196861,1741147,101322779,North America,37.090240,-95.712891
2,215353593,36824580,208134,35919372,South America,-14.235004,-51.925280
3,65584518,39524311,95532,39264546,Europe,46.227638,2.213749
4,60756135,4055966,40865,3912506,Africa,-30.559482,22.937506


In [8]:
query = """
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
               Country,
               Continent
                        
            From 
                covid
            Order by 
                "Total Cases" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head(10)

,Total Cases,Active Cases,Total Recovered,Country,Continent
0,104196861,1741147,101322779,United States,North America
1,44682784,1755,44150289,India,Asia
2,39524311,95532,39264546,France,Europe
3,37779833,216022,37398100,Germany,Europe
4,36824580,208134,35919372,Brazil,South America
5,32588442,10952618,21567425,Japan,Asia
6,30197066,422703,29740877,S. Korea,Asia
7,25453789,251970,25014986,Italy,Europe
8,24274361,50102,24020088,UK,Europe
9,21958696,207580,21356008,Russia,Asia


In [9]:
query = """
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
               Country,
               Continent,
               Population
                        
            From 
                covid
            Order by 
                "Population" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head(10)

,Total Cases,Active Cases,Total Recovered,Country,Continent,Population
0,503302,118977,379053,China,Asia,1412000000
1,44682784,1755,44150289,India,Asia,1406631776
2,104196861,1741147,101322779,United States,North America,334805269
3,6730289,4264,6565208,Indonesia,Asia,279134505
4,1576313,6984,1538689,Pakistan,Asia,229488994
5,266463,3458,259850,Nigeria,Africa,216746934
6,36824580,208134,35919372,Brazil,South America,215353593
7,2037556,15420,1992694,Bangladesh,Asia,167885689
8,21958696,207580,21356008,Russia,Asia,145805947
9,7368252,429421,6606633,Mexico,North America,131562772


In [10]:
# user inputs
user_continent = 'North America'
user_min_total_cases = 10000

# switch on user_region
if user_continent == 'All':
    where_clause = " 1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
                "Total Deaths",
                Continent,
                Country,
                Population
                        
            From 
                covid
            Where
                "Total Cases" >= {user_min_total_cases} AND
               {where_clause}
            Order by 
                "Total Cases" DESC;
            
            """
df_sunburst = pd.read_sql(text(query), con=engine)
df_sunburst.head()
# df3.info()

,Total Cases,Active Cases,Total Recovered,Total Deaths,Continent,Country,Population
0,104196861,1741147,101322779,1132935,North America,United States,334805269
1,7368252,429421,6606633,332198,North America,Mexico,131562772
2,4550256,55863,4444013,50380,North America,Canada,38388419
3,1227853,3202,1204545,20106,North America,Guatemala,18584039
4,1186176,316307,860711,9158,North America,Costa Rica,5182354


In [11]:
# user inputs
user_continent = 'North America'
user_min_total_cases = 10000

# switch on user_region
if user_continent == 'All':
    where_clause = "1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT
                "Population",
                "Total Test",
                "Total Cases",
                "Active Cases",
                "Total Deaths",
                "Total Recovered",
                Continent,
                Country
                                      
            From 
                covid
            Where
                "Total Cases" >= {user_min_total_cases} AND
               {where_clause}
            Order by 
                "Total Cases" DESC;
            
            """
df_table = pd.read_sql(text(query), con=engine)
df_table.head()
# df3.info()

,Population,Total Test,Total Cases,Active Cases,Total Deaths,Total Recovered,Continent,Country
0,334805269,1159832679,104196861,1741147,1132935,101322779,North America,United States
1,131562772,19356195,7368252,429421,332198,6606633,North America,Mexico
2,38388419,66343123,4550256,55863,50380,4444013,North America,Canada
3,18584039,6757892,1227853,3202,20106,1204545,North America,Guatemala
4,5182354,4659757,1186176,316307,9158,860711,North America,Costa Rica


In [12]:
# user inputs
user_continent = 'North America'
user_min_total_cases = 10000

# switch on user_region
if user_continent == 'All':
    where_clause = "1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT
                "Total Cases",
                "Total Deaths",
                Continent,
                Country
                                      
            From 
                covid
            Where
                "Total Cases" >= {user_min_total_cases} AND
               {where_clause}
            Order by 
                "Total Cases" DESC;
            
            """
df_stack = pd.read_sql(text(query), con=engine)
df_stack.head()
# df3.info()

,Total Cases,Total Deaths,Continent,Country
0,104196861,1132935,North America,United States
1,7368252,332198,North America,Mexico
2,4550256,50380,North America,Canada
3,1227853,20106,North America,Guatemala
4,1186176,9158,North America,Costa Rica


In [13]:
# user inputs
user_continent = 'North America'
user_min_total_cases = 10000

# switch on user_region
if user_continent == 'All':
    where_clause = "1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT
                "Population",
                "Total Test",
                "Total Cases",
                "Active Cases",
                "Total Deaths",
                "Total Recovered",
                Continent,
                Country
                                      
            From 
                covid
            Where
                "Total Cases" >= {user_min_total_cases} AND
               {where_clause}
            Order by 
                "Total Cases" DESC;
            
            """
df_map = pd.read_sql(text(query), con=engine)
df_map.head()
# df3.info()

,Population,Total Test,Total Cases,Active Cases,Total Deaths,Total Recovered,Continent,Country
0,334805269,1159832679,104196861,1741147,1132935,101322779,North America,United States
1,131562772,19356195,7368252,429421,332198,6606633,North America,Mexico
2,38388419,66343123,4550256,55863,50380,4444013,North America,Canada
3,18584039,6757892,1227853,3202,20106,1204545,North America,Guatemala
4,5182354,4659757,1186176,316307,9158,860711,North America,Costa Rica


In [25]:
# user inputs
user_continent = 'All'
user_min_total_cases = 1

# switch on user_region
if user_continent == 'All':
    where_clause = "1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT
               Continent,
                SUM("Population") AS Population,
                SUM("Total Test") AS "Total Test",
                SUM("Total Cases") AS "Total Cases",
                SUM("Active Cases") AS "Active Cases",
                SUM("Total Deaths") AS "Total Deaths",
                SUM("Total Recovered") AS "Total Recovered"
                

            From 
                covid
            Group by
                "Continent"
            Order by 
                "Total Cases" DESC;
            
            """
                # AVG(TTest_TCases_Ratio) AS TT_to_TC,
                # AVG(TTest_TDeath_Ratio) AS TT_to_TD,
                # AVG(TTest_Pop_Ratio) AS TT_to_P,
                # AVG(TCases_TReco_Ratio) AS TC_to_TR
df_table = pd.read_sql((query), con=engine)
# Round the AVG columns to 2 decimal places
# avg_columns = ["TT_to_TC", "TT_to_TD", "TT_to_P", "TC_to_TR"]
# df_table[avg_columns] = df_table[avg_columns].round(2)
df_table.head(12)


,Continent,Population,Total Test,Total Cases,Active Cases,Total Deaths,Total Recovered
0,Europe,620787732,2583501791,226805930,2049453,1648343,223108134
1,Asia,4758797868,2599433470,226035039,14051632,1886525,210096881
2,North America,579867720,1290777252,122271405,2648949,1582497,118039959
3,South America,430088015,239661135,66914766,394857,1326749,65193160
4,Oceania,42149054,88438422,13739096,72144,24338,13642614
5,Africa,1320243661,103105170,10985120,292287,226785,10466048


In [23]:
df_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Continent         6 non-null      object
 1   TotalPopulation   6 non-null      int64 
 2   TotalTests        6 non-null      int64 
 3   TotalCases        6 non-null      int64 
 4   TotalActiveCases  6 non-null      int64 
 5   TotalDeaths       6 non-null      int64 
 6   TotalRecovered    6 non-null      int64 
dtypes: int64(6), object(1)
memory usage: 464.0+ bytes


In [27]:
df_table['TotalTest_TotalCases_Ratio'] = df_table['Total Test'] / df_table['Total Cases']
df_table['TotalTest_TotalDeath_Ratio'] = df_table['Total Test'] / df_table['Total Deaths']
df_table['TotalTest_Population_Ratio'] = df_table['Total Test'] / df_table['Population']
df_table['TotalCases_Recovered_Ratio'] = df_table['Total Cases'] / df_table['Total Recovered']
df_table['TotalTest_TotalCases_Ratio'] = df_table['TotalTest_TotalCases_Ratio'].round(2)
df_table['TotalTest_TotalDeath_Ratio'] = df_table['TotalTest_TotalDeath_Ratio'].round(2)
df_table['TotalTest_Population_Ratio'] = df_table['TotalTest_Population_Ratio'].round(2)
df_table['TotalCases_Recovered_Ratio']  =df_table['TotalCases_Recovered_Ratio'] .round(2)
df_table.head()

,Continent,Population,Total Test,Total Cases,Active Cases,Total Deaths,Total Recovered,TotalTest_TotalCases_Ratio,TotalTest_TotalDeath_Ratio,TotalTest_Population_Ratio,TotalCases_Recovered_Ratio
0,Europe,620787732,2583501791,226805930,2049453,1648343,223108134,11.39,1567.33,4.16,1.02
1,Asia,4758797868,2599433470,226035039,14051632,1886525,210096881,11.50,1377.90,0.55,1.08
2,North America,579867720,1290777252,122271405,2648949,1582497,118039959,10.56,815.66,2.23,1.04
3,South America,430088015,239661135,66914766,394857,1326749,65193160,3.58,180.64,0.56,1.03
4,Oceania,42149054,88438422,13739096,72144,24338,13642614,6.44,3633.76,2.10,1.01


In [34]:
selected_df = df_table[["Continent","TotalTest_TotalCases_Ratio", "TotalTest_TotalDeath_Ratio","TotalTest_Population_Ratio","TotalCases_Recovered_Ratio"]]
selected_df = selected_df.sort_values(by="TotalTest_TotalDeath_Ratio", ascending=False)
selected_df.head()


,Continent,TotalTest_TotalCases_Ratio,TotalTest_TotalDeath_Ratio,TotalTest_Population_Ratio,TotalCases_Recovered_Ratio
4,Oceania,6.44,3633.76,2.10,1.01
0,Europe,11.39,1567.33,4.16,1.02
1,Asia,11.50,1377.90,0.55,1.08
2,North America,10.56,815.66,2.23,1.04
5,Africa,9.39,454.64,0.08,1.05


In [27]:
# user inputs
user_continent = 'All'
user_min_total_cases = 1

# switch on user_region
if user_continent == 'All':
    where_clause = "1=1"
else:
    where_clause = f"Continent = '{user_continent}'"

query = f"""
            SELECT
               Continent,
                "Population" AS TotalPopulation,
                "Total Test" AS TotalTests,
                "Total Cases" AS TotalCases,
                "Active Cases" AS TotalActiveCases,
                "Total Deaths" AS TotalDeaths,
                "Total Recovered" AS TotalRecovered,
                Country
                

            From 
                covid
            Order by 
                "Total Cases" DESC;
            
            """
df_table_all = pd.read_sql((query), con=engine)
df_table_all.head(12)


,Continent,TotalPopulation,TotalTests,TotalCases,TotalActiveCases,TotalDeaths,TotalRecovered,Country
0,North America,334805269,1159832679,104196861,1741147,1132935,101322779,United States
1,Asia,1406631776,915265788,44682784,1755,530740,44150289,India
2,Europe,65584518,271490188,39524311,95532,164233,39264546,France
3,Europe,83883596,122332384,37779833,216022,165711,37398100,Germany
4,South America,215353593,63776166,36824580,208134,697074,35919372,Brazil
5,Asia,125584838,92144639,32588442,10952618,68399,21567425,Japan
6,Asia,51329899,15804065,30197066,422703,33486,29740877,S. Korea
7,Europe,60262770,265478247,25453789,251970,186833,25014986,Italy
8,Europe,68497907,522526476,24274361,50102,204171,24020088,UK
9,Asia,145805947,273400000,21958696,207580,395108,21356008,Russia


In [24]:
df_table_all['TotalTest_TotalCases_Ratio'] = df_table_all['TotalTests'] / df_table_all['TotalCases']
df_table_all['TotalTest_TotalDeath_Ratio'] = df_table_all['TotalTests'] / df_table_all['TotalDeaths']
df_table_all['TotalTest_Population_Ratio'] = df_table_all['TotalTests'] / df_table_all['TotalPopulation']
df_table_all['TotalTest_TotalCases_Ratio'] = df_table_all['TotalTest_TotalCases_Ratio'].round(2)
df_table_all['TotalTest_TotalDeath_Ratio'] = df_table_all['TotalTest_TotalDeath_Ratio'].round(2)
df_table_all['TotalTest_Population_Ratio'] = df_table_all['TotalTest_Population_Ratio'].round(2)
df_table_all.head()

,Continent,TotalPopulation,TotalTests,TotalCases,TotalActiveCases,TotalDeaths,TotalRecovered,TotalTest_TotalCases_Ratio,TotalTest_TotalDeath_Ratio,TotalTest_Population_Ratio
0,North America,334805269,1159832679,104196861,1741147,1132935,101322779,11.13,1023.74,3.46
1,Asia,1406631776,915265788,44682784,1755,530740,44150289,20.48,1724.51,0.65
2,Europe,65584518,271490188,39524311,95532,164233,39264546,6.87,1653.08,4.14
3,Europe,83883596,122332384,37779833,216022,165711,37398100,3.24,738.23,1.46
4,South America,215353593,63776166,36824580,208134,697074,35919372,1.73,91.49,0.30
